In [ ]:
# https://www.kaggle.com/hakeem/stacked-then-averaged-models-0-5697/code 

In [1]:
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin, ClassifierMixin
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb
import pandas as pd
from sklearn.linear_model import ElasticNetCV, LassoLarsCV
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.pipeline import make_pipeline, make_union
from sklearn.utils import check_array
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn.random_projection import GaussianRandomProjection, SparseRandomProjection
from sklearn.decomposition import PCA, FastICA
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import r2_score

/home/manish/anaconda2/envs/package15/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [46]:
from sklearn.ensemble import RandomForestRegressor

In [40]:
class StackingEstimator(BaseEstimator, TransformerMixin):
    
    def __init__(self, estimator):
        self.estimator = estimator

    def fit(self, X, y=None, **fit_params):
        self.estimator.fit(X, y, **fit_params)
        return self
    def transform(self, X):
        X = check_array(X)
        X_transformed = np.copy(X)
        # add class probabilities as a synthetic feature
        if issubclass(self.estimator.__class__, ClassifierMixin) and hasattr(self.estimator, 'predict_proba'):
            X_transformed = np.hstack((self.estimator.predict_proba(X), X))

        # add class prodiction as a synthetic feature
        X_transformed = np.hstack((np.reshape(self.estimator.predict(X), (-1, 1)), X_transformed))

        return X_transformed


In [21]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [4]:
train.head()

,ID,y,X0,X1,X2,X3,X4,X5,X6,X8,...,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
0,0,130.81,k,v,at,a,d,u,j,o,...,0,0,1,0,0,0,0,0,0,0
1,6,88.53,k,t,av,e,d,y,l,o,...,1,0,0,0,0,0,0,0,0,0
2,7,76.26,az,w,n,c,d,x,j,x,...,0,0,0,0,0,0,1,0,0,0
3,9,80.62,az,t,n,f,d,x,l,e,...,0,0,0,0,0,0,0,0,0,0
4,13,78.02,az,v,n,f,d,h,d,n,...,0,0,0,0,0,0,0,0,0,0


In [22]:
for c in train.columns:
    if train[c].dtype == 'object':
        lbl = LabelEncoder()
        lbl.fit(list(train[c].values) + list(test[c].values))
        train[c] = lbl.transform(list(train[c].values))
        test[c] = lbl.transform(list(test[c].values))

In [6]:
train.head()

,ID,y,X0,X1,X2,X3,X4,X5,X6,X8,...,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
0,0,130.81,37,23,20,0,3,27,9,14,...,0,0,1,0,0,0,0,0,0,0
1,6,88.53,37,21,22,4,3,31,11,14,...,1,0,0,0,0,0,0,0,0,0
2,7,76.26,24,24,38,2,3,30,9,23,...,0,0,0,0,0,0,1,0,0,0
3,9,80.62,24,21,38,5,3,30,11,4,...,0,0,0,0,0,0,0,0,0,0
4,13,78.02,24,23,38,5,3,14,3,13,...,0,0,0,0,0,0,0,0,0,0


In [23]:
n_comp = 12

In [24]:
tsvd = TruncatedSVD(n_components=n_comp, random_state=420)
tsvd_results_train = tsvd.fit_transform(train.drop(['y'],axis=1))
tsvd_results_test = tsvd.transform(test)

In [10]:
pca = PCA(n_components=n_comp, random_state=420)
pca2_results_train = pca.fit_transform(train.drop(['y'],axis=1))
pca_results_test = pca.fit_transform(test)

In [11]:
ica = FastICA(n_components=n_comp, random_state=420)
ica2_results_train = ica.fit_transform(train.drop(['y'],axis=1))
ica2_results_test = ica.fit_transform(test)

In [12]:
grp = GaussianRandomProjection(n_components=n_comp, eps=0.1, random_state=420)
grp_results_train = grp.fit_transform(train.drop(['y'],axis=1))
grp_results_test = grp.fit_transform(test)

In [13]:
srp = SparseRandomProjection(n_components=n_comp, random_state=420)
srp_results_train = srp.fit_transform(train.drop(['y'],axis=1))
srp_results_test = srp.fit_transform(test)

In [14]:
usable_columns = list(set(train.columns) - set(['y'])a)

In [25]:
for i in range(1, n_comp + 1):
    train['pca_' + str(i)] = pca2_results_train[:, i - 1]
    test['pca_' + str(i)] = pca_results_test[:, i - 1]

    train['ica_' + str(i)] = ica2_results_train[:, i - 1]
    test['ica_' + str(i)] = ica2_results_test[:, i - 1]

    train['tsvd_' + str(i)] = tsvd_results_train[:, i - 1]
    test['tsvd_' + str(i)] = tsvd_results_test[:, i - 1]

    train['grp_' + str(i)] = grp_results_train[:, i - 1]
    test['grp_' + str(i)] = grp_results_test[:, i - 1]

    train['srp_' + str(i)] = srp_results_train[:, i - 1]
    test['srp_' + str(i)] = srp_results_test[:, i - 1]

In [26]:
train.head()

,ID,y,X0,X1,X2,X3,X4,X5,X6,X8,...,pca_11,ica_11,tsvd_11,grp_11,srp_11,pca_12,ica_12,tsvd_12,grp_12,srp_12
0,0,130.81,37,23,20,0,3,27,9,14,...,1.869482,-0.009489,1.593338,-18.037091,-1.272022,0.577363,0.021648,0.119684,-12.484118,33.072579
1,6,88.53,37,21,22,4,3,31,11,14,...,-0.034246,0.000106,0.840350,-14.242336,-3.816067,0.901663,0.012759,-0.878237,-10.781389,36.888646
2,7,76.26,24,24,38,2,3,30,9,23,...,-0.189681,-0.001965,0.836276,-8.944316,-3.816067,-0.998406,-0.022951,0.657464,-13.076789,35.616624
3,9,80.62,24,21,38,5,3,30,11,4,...,-0.650456,-0.005130,-0.061997,-8.104063,-5.088089,-1.368278,-0.022663,0.848731,-8.853840,36.888646
4,13,78.02,24,23,38,5,3,14,3,13,...,-0.729887,-0.010211,-1.283128,-3.322663,-3.816067,-1.617561,-0.039583,1.096843,-22.018516,15.264267


In [27]:
y_train = train.y.values
y_mean = np.mean(y_train)
id_test = test['ID'].values

In [28]:
finaltrainset = train[usable_columns].values
finaltestset = test[usable_columns].values

In [33]:
'''Train the xgb model then predict the test data'''

xgb_params = {
    'n_trees': 520, 
    'eta': 0.0045,
    'max_depth': 4,
    'subsample': 0.93,
    'objective': 'reg:linear',
    'eval_metric': 'rmse',
    'base_score': y_mean, # base prediction = mean(target)
    'silent': 1
}
# NOTE: Make sure that the class is labeled 'class' in the data file

dtrain = xgb.DMatrix(train.drop('y', axis=1), y_train)
dtest = xgb.DMatrix(test)

num_boost_rounds = 1250
# train model
model = xgb.train(dict(xgb_params,silent=0), dtrain, num_boost_round=num_boost_rounds)

In [34]:
y_pred = model.predict(dtest)

In [48]:
stacked_pipeline = make_pipeline(
    StackingEstimator(estimator=LassoLarsCV(normalize=True)),
    StackingEstimator(estimator=GradientBoostingRegressor(n_estimators=500,learning_rate=0.001, loss="huber", max_depth=3, max_features=0.55, min_samples_leaf=18, min_samples_split=14, subsample=0.7)),
    StackingEstimator(estimator=RandomForestRegressor(n_estimators=500,max_depth=3,min_samples_leaf=18,min_samples_split=14)),
    LassoLarsCV()

)

In [49]:
stacked_pipeline.fit(finaltrainset, y_train)
results = stacked_pipeline.predict(finaltestset)

/home/manish/anaconda2/envs/package15/lib/python2.7/site-packages/sklearn/linear_model/least_angle.py:309: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 18 iterations, i.e. alpha=3.727e-03, with an active set of 18 regressors, and the smallest cholesky pivot element being 1.490e-08
  ConvergenceWarning)
/home/manish/anaconda2/envs/package15/lib/python2.7/site-packages/sklearn/linear_model/least_angle.py:309: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 48 iterations, i.e. alpha=2.524e-03, with an active set of 46 regressors, and the smallest cholesky pivot element being 1.490e-08
  ConvergenceWarning)
/home/manish/anaconda2/envs/package15/lib/python2.7/site-packages/sklearn/linear_model/least_angle.py:309: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 56 iterations, i.e. alpha=2.264e-03, with an active set of 54 regressors, and the smallest cholesky pivot element being 

In [50]:
print('R2 score on train data:')
print(r2_score(y_train,stacked_pipeline.predict(finaltrainset)*0.2855 + model.predict(dtrain)*0.7145))

R2 score on train data:
0.661534214061


In [51]:
sub = pd.DataFrame()
sub['ID'] = id_test
sub['y'] = y_pred*0.75 + results*0.25
sub.to_csv('three_offline_stacked-models.csv', index=False)


In [52]:
sub.head()

,ID,y
0,1,90.388698
1,2,101.438360
2,3,82.682462
3,4,81.305126
4,5,115.929837
